In [72]:
#todo
#statistics of returns
#regression (?)
#improve charts (titles etc..)
#import financials, possibile in a new "tab"
#import twitter scraping for sentimetn
#additional tab with portfolio (?)

env: ALPHAVANTAGE_API_KEY=NTN3D6P6LIMZ6A2Q#alpha advantage api key
Dash app running on http://127.0.0.1:8050/


In [28]:
#graph imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
#data imports
import pandas as pd
import numpy as np
import os
import re
import pandas_datareader.data as web
from datetime import datetime as dt
from bs4 import BeautifulSoup
import requests
import sys
################################################################################
%env ALPHAVANTAGE_API_KEY = NTN3D6P6LIMZ6A2Q#alpha advantage api key
################################################################################


from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='YOUR_API_KEY',output_format='pandas', indexing_type='date')

################################################################################
#adjust value function
################################################################################
def adjust(date, close, adj_close, in_col, rounding=4):
    '''
    If using forex or Crypto - Change the rounding accordingly!
    '''
    try:
        factor = adj_close / close
        return round(in_col * factor, rounding)
    except ZeroDivisionError:
        print('WARNING: DIRTY DATA >> {} Close: {} | Adj Close {} | in_col: {}'.format(date, close, adj_close, in_col))
        return 0
    
    
def drawText():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2("Text"),
                ], style={'textAlign': 'center'}) 
            ])
        ),
    ])

app = JupyterDash(external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div([       
                    html.Div([
                        dbc.Card(
                            dbc.CardBody([
                                html.Div([
                                    html.H1('Stock Dashboard')
                                ],style={'textAlign': 'center'})                                
                            ])
                        )
                    ]),
                    html.Br(),
    
                    html.Div([
                        dbc.CardDeck(
                            dbc.Card(
                                dbc.CardBody([
                                    html.Div([
                                        html.H2('Insert a stock ticker'),
                                        dcc.Input(id = 'stock-ticker', value = 'KO', type = 'text'),
                                        html.Button(id = 'submit-ticker', n_clicks = 0, children = 'Submit'),
                                        dcc.Store(id='stock-storage'),
                                    ])
                                ]),
                            ),
                            dbc.Card(
                                dbc.CardBody([
                                    
                                ])
                            ),
                        )
                    ]),
                    
                    
                    html.Div(id = 'stock-name'),    
                    html.Div(id = 'stock-price'),    
                    html.Div([dcc.DatePickerRange(id="my-date-picker-range",
                                                    min_date_allowed=dt(2000, 1, 1),
                                                    max_date_allowed=dt.today(),
                                                    start_date = dt(2000, 1, 1),
                                                    end_date=dt.today(),
                                                    updatemode='bothdates')]),
                    html.Br(),
                    html.Div(id = 'candlestick-graph'),
                    html.Div(id = 'daily-returns-graph'),
                    html.Br(),
                    html.Div(id = 'cum-returns-graph'),
                    html.Div(id = 'daily-returns-time-graph'),
                    html.Br(),
                    html.Div(id = 'bs-annual'),
                    html.Div(id = 'bs-quarterly'),
])

@app.callback(Output(component_id='stock-storage', component_property='data'),
              Output(component_id='stock-name', component_property='children'),
              Output(component_id='stock-price', component_property='children'),
              Output(component_id='bs-annual', component_property = 'children'),
              Output(component_id='bs-quarterly', component_property = 'children'),
              Input('submit-ticker', 'n_clicks'),
              State('stock-ticker', 'value'))
def update_output(n_clicks,stock):
    df, meta_data = ts.get_daily_adjusted(stock, outputsize='full')
    df.sort_index(ascending=True, inplace = True)  
    
    url = "https://finance.yahoo.com/quote/{}".format(stock)

    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')

    price = soup.find('span', class_='Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)')
    name = soup.find('h1', class_ = 'D(ib) Fz(18px)')
    
    #get company Balance Sheet
    #understand how to use environmental variable
    url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={}&apikey=NTN3D6P6LIMZ6A2Q'.format(stock)
    r = requests.get(url)
    data = r.json()
    #Annual Reports  
    dfAR = pd.json_normalize(data, record_path = 'annualReports')
    dfAR = dfAR.set_index(dfAR['fiscalDateEnding'])
    dfAR.drop(["fiscalDateEnding"], axis = 1, inplace = True)
    dfAR = dfAR.transpose()
    dfAR.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
    dfAR.reset_index(inplace= True)
    for i in list(dfAR.keys()):
        for j in list(dfAR[i].index):
            try:
                dfAR[i][j] = float(dfAR[i][j])/1000000000
            except:
                dfAR[i][j] = dfAR[i][j]
    #Quarterly Reports            
    dfQR = pd.json_normalize(data, record_path = 'quarterlyReports')
    dfQR = dfQR.set_index(dfQR['fiscalDateEnding'])
    dfQR.drop(["fiscalDateEnding"], axis = 1, inplace = True)
    dfQR = dfQR.transpose()
    dfQR.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
    dfQR.reset_index(inplace= True)
    for i in list(dfQR.keys()):
        for j in list(dfQR[i].index):
            try:
                dfQR[i][j] = float(dfQR[i][j])/1000000000
            except:
                dfQR[i][j] = dfQR[i][j]
                
    bs_annual = dash_table.DataTable(id='balance-sheet-annualy',
                                      columns=[{"name": i, "id": i} for i in dfAR.columns],
                                      data=dfAR.to_dict('records'),
                                      style_header={'backgroundColor': 'rgb(30, 30, 30)'},    
                                      style_cell={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'white' },
                                      style_table={'overflowX': 'auto'},
                                     )
    bs_quarterly = dash_table.DataTable(id='balance-sheet-quarterly',
                                      columns=[{"name": i, "id": i} for i in dfQR.columns],
                                      data=dfQR.to_dict('records'),
                                      style_header={'backgroundColor': 'rgb(30, 30, 30)'},    
                                      style_cell={'backgroundColor': 'rgb(50, 50, 50)', 'color': 'white' },
                                      style_table={'overflowX': 'auto'},
                                     )
#     #Annual Earnings 
#     #understand how to use environmental variable
#     url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol={}&apikey=NTN3D6P6LIMZ6A2Q'.format(stock)
#     r = requests.get(url)
#     data = r.json()
#     dfAE = pd.json_normalize(data, record_path = 'annualEarnings')
#     dfAE = dfAE.set_index(dfAE['fiscalDateEnding'])
#     dfAE.drop(["fiscalDateEnding"], axis = 1, inplace = True)
#     dfAE = dfAE.transpose()
#     dfAE.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)    
#     #Quarterly Earnings
#     dfQE = pd.json_normalize(data, record_path = 'quarterlyEarnings')
#     dfQE = dfQE.set_index(dfQE['fiscalDateEnding'])
#     dfQE.drop(["fiscalDateEnding"], axis = 1, inplace = True)
#     dfQE = dfQE.transpose()
#     dfQE.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
    
    
    
#     #Annual Income Statement
#     url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={}&apikey=NTN3D6P6LIMZ6A2Q'.format(stock)
#     r = requests.get(url)
#     data = r.json()
#     dfAIS = pd.json_normalize(data, record_path = 'annualReports')
#     dfAIS = dfAIS.set_index(dfAIS['fiscalDateEnding'])
#     dfAIS.drop(["fiscalDateEnding"], axis = 1, inplace = True)
#     dfAIS = dfAIS.transpose()
#     dfAIS.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
#     for i in list(dfAIS.keys()):
#         for j in list(dfAIS[i].index):
#             try:
#                 dfAIS[i][j] = float(dfAIS[i][j])/1000000000
#             except:
#                 dfAIS[i][j] = dfAIS[i][j]
#     #Quarterly Income Statement
#     dfQIS = pd.json_normalize(data, record_path = 'quarterlyReports')
#     dfQIS = dfQIS.set_index(dfQIS['fiscalDateEnding'])
#     dfQIS.drop(["fiscalDateEnding"], axis = 1, inplace = True)
#     dfQIS = dfQIS.transpose()
#     dfQIS.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
#     for i in list(dfQIS.keys()):
#         for j in list(dfQIS[i].index):
#             try:
#                 dfQIS[i][j] = float(dfQIS[i][j])/1000000000
#             except:
#                 dfQIS[i][j] = dfQIS[i][j]
    
    
#     #Annual Cash Flow
#     url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={}&apikey=NTN3D6P6LIMZ6A2Q'.format(stock)
#     r = requests.get(url)
#     data = r.json()
#     dfACF = pd.json_normalize(data, record_path = 'annualReports')
#     dfACF = dfACF.set_index(dfACF['fiscalDateEnding'])
#     dfACF.drop(["fiscalDateEnding"], axis = 1, inplace = True)
#     dfACF = dfACF.transpose()
#     dfACF.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
#     for i in list(dfACF.keys()):
#         for j in list(dfACF[i].index):
#             try:
#                 dfACF[i][j] = float(dfACF[i][j])/1000000000
#             except:
#                 dfACF[i][j] = dfACF[i][j]
#     #Quarterly Cash Flow
#     dfQCF = pd.json_normalize(data, record_path = 'quarterlyReports')
#     dfQCF = dfQCF.set_index(dfQCF['fiscalDateEnding'])
#     dfQCF.drop(["fiscalDateEnding"], axis = 1, inplace = True)
#     dfQCF = dfQCF.transpose()
#     dfQCF.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
#     for i in list(dfQCF.keys()):
#         for j in list(dfQCF[i].index):
#             try:
#                 dfQCF[i][j] = float(dfQCF[i][j])/1000000000
#             except:
#                 dfQCF[i][j] = dfQCF[i][j]  

#     #convet back df to json? how to display df as a table?
    
    return df.to_json(),  html.H2(name.text), html.H2("Last price: {}".format(price.text)), bs_annual, bs_quarterly
                                    

@app.callback(Output(component_id='daily-returns-graph' , component_property= 'children'),
              Output(component_id='candlestick-graph', component_property= 'children'),
              Output(component_id='cum-returns-graph' , component_property= 'children'),
              Output(component_id='daily-returns-time-graph',component_property='children'),
              Output(component_id='my-date-picker-range', component_property = 'start_date'),
               Output(component_id='my-date-picker-range', component_property = 'end_date'),
              Input('stock-storage','data') )
def update_chart(stock):
    if stock is not None:
        df = pd.read_json(stock)

        df['adj close'] = df['5. adjusted close']
        df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
        df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
        df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])
    
    
        #calculate daily returns
        df['daily_return'] = df['adj close'].pct_change()
        df['log_returns'] = (np.log(df['adj close'] /    df['adj close'].shift(-1)))
        # calculate cumluative return
        df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000
    
        daily_returns = dcc.Graph(#id='daily-returns-graph',
                                        figure=px.histogram(df, x="daily_return",  title = 'Daily returns distribution'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Returns",
                                            yaxis_title="Frequency",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )   
        candlestick = dcc.Graph(#id='candlestick-graph',
                                        figure=go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['adj open'],
                                                        high=df['adj high'],
                                                        low=df['adj low'],
                                                        close=df['adj close'])]
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        cum_returns = dcc.Graph(#id='cum-returns-graph',
                                        figure=px.line(df, x=df.index,y="cumluative_return", title = 'Cumulative returns'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Value of portfolio",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        daily_returns_time = dcc.Graph(#id='daily-returns-time-graph',
                                        figure=px.line(df, x=df.index,y="daily_return", title = 'Daily returns over time'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Returns",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )            
        date_picker = dcc.DatePickerRange(id = 'my-date-picker-range',
                                min_date_allowed=pd.to_datetime(df.index).min(),
                                max_date_allowed=pd.to_datetime(df.index).max(),
                                start_date=pd.to_datetime(df.index).min(),
                                end_date=pd.to_datetime(df.index).max(),
                                updatemode='bothdates')
    
        return daily_returns, candlestick,  cum_returns, daily_returns_time, pd.to_datetime(df.index).min(), pd.to_datetime(df.index).max()
    else:
        return None, None, None, None, None, None


@app.callback(Output(component_id='daily-returns-graph' , component_property= 'children'),
              Output(component_id='candlestick-graph', component_property= 'children'),
              Output(component_id='cum-returns-graph' , component_property= 'children'),
              Output(component_id='daily-returns-time-graph',component_property='children'),
              Input('my-date-picker-range', 'start_date'),
              Input('my-date-picker-range', 'end_date'),
              Input('stock-storage','data') )
def update_chart_time(start, end, stock):
    if stock is not None and end is not None and start is not None:
        df = pd.read_json(stock)
        df = df[start:end]#verificare per fare entrare giorno corrente
        df['adj close'] = df['5. adjusted close']
        df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
        df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
        df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])


        #calculate daily returns
        df['daily_return'] = df['adj close'].pct_change()
        df['log_returns'] = (np.log(df['adj close'] /    df['adj close'].shift(-1)))
        # calculate cumluative return
        df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000
    
        daily_returns = dcc.Graph(#id='daily-returns-graph',
                                        figure=px.histogram(df, x="daily_return",  title = 'Daily returns distribution'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Returns",
                                            yaxis_title="Frequency",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )   
        candlestick = dcc.Graph(#id='candlestick-graph',
                                        figure=go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['adj open'],
                                                        high=df['adj high'],
                                                        low=df['adj low'],
                                                        close=df['adj close'])]
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        cum_returns = dcc.Graph(#id='cum-returns-graph',
                                        figure=px.line(df, x=df.index,y="cumluative_return", title = 'Cumulative returns'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Value of protfolio",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        daily_returns_time = dcc.Graph(#id='daily-returns-time-graph',
                                        figure=px.line(df, x=df.index,y="daily_return", title = 'Daily returns over time'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Returns",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )            
    
    
        return daily_returns, candlestick,  cum_returns, daily_returns_time
    else:
        return None,None,None,None


# Run app and display result inline in the notebook
app.run_server(mode="external", port = '8060' )
#mode="external", host="127.0.0.1", port=8060

env: ALPHAVANTAGE_API_KEY=NTN3D6P6LIMZ6A2Q#alpha advantage api key


TypeError: The `dash_bootstrap_components.CardDeck` component (version 0.12.0) with the ID "Card(CardBody([]))" detected a Component for a prop other than `children`
Did you forget to wrap multiple `children` in an array?
Prop id has value Card(CardBody([]))


In [12]:
df, meta_data = ts.get_daily_adjusted('SPOT', outputsize='full')
df.sort_index(ascending=True, inplace = True)  
df['adj close'] = df['5. adjusted close']
df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])
    
    
#calculate daily returns
df['daily_return'] = df['adj close'].pct_change()
# calculate cumluative return
df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000